# Exon-Level Reference GTF File Generation

This guide explains how to generate an exon-level GTF reference file. This file is used to align scRNA-seq data to the exon level, allowing the extraction of exon read counts and junction read counts. The goal of the exon-level GTF is to ensure that exons within each gene are unique and do not overlap with one another.

![This is an example image](./exon_gtf_demonstration.png)


### **For Human GRCh38**
You can directly download the pre-generated exon-level GTF file from [here](https://mcgill-my.sharepoint.com/my?id=%2Fpersonal%2Fkailu%5Fsong%5Fmail%5Fmcgill%5Fca%2FDocuments%2FDeepExonas%5Fgithub%5Fexample%2Falignment%5Ffiles).  

### **For Other Species**
1. First, download the reference GTF file from [here](https://www.ensembl.org/index.html).  

2. Then, run this script to generate the exon-level GTF file.

In [1]:
#import annotation file
import pandas as pd
from DOLPHIN import gtfpy
import numpy as np
import math

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',100)

/mnt/md0/kailu/anaconda3/envs/DOLPHIN/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#load the original ensembl gtf file
df_gtf=gtfpy.readGTF("Homo_sapiens.GRCh38.107.gtf")
GTFpa=gtfpy.parseGTF(df_gtf)

In [5]:
GTFpa

,seqname,source,feature,start,end,score,strand,frame,gene_id,protein_id,gene_source,exon_number,gene_biotype,transcript_version,transcript_support_level,exon_version,exon_id,transcript_source,transcript_name,gene_name,tag,ccds_id,transcript_biotype,transcript_id,protein_version,gene_version
0,1,ensembl_havana,gene,1471765,1497848,.,+,.,ENSG00000160072,NaN,ensembl_havana,NaN,protein_coding,NaN,NaN,NaN,NaN,NaN,NaN,ATAD3B,NaN,NaN,NaN,NaN,NaN,20
1,1,ensembl_havana,transcript,1471765,1497848,.,+,.,ENSG00000160072,NaN,ensembl_havana,NaN,protein_coding,1,NaN,NaN,NaN,ensembl_havana,ATAD3B-206,ATAD3B,basic,CCDS30,protein_coding,ENST00000673477,NaN,20
2,1,ensembl_havana,exon,1471765,1472089,.,+,.,ENSG00000160072,NaN,ensembl_havana,1,protein_coding,1,NaN,1,ENSE00003889014,ensembl_havana,ATAD3B-206,ATAD3B,basic,CCDS30,protein_coding,ENST00000673477,NaN,20
3,1,ensembl_havana,CDS,1471885,1472089,.,+,0,ENSG00000160072,ENSP00000500094,ensembl_havana,1,protein_coding,1,NaN,NaN,NaN,ensembl_havana,ATAD3B-206,ATAD3B,basic,CCDS30,protein_coding,ENST00000673477,1,20
4,1,ensembl_havana,start_codon,1471885,1471887,.,+,0,ENSG00000160072,NaN,ensembl_havana,1,protein_coding,1,NaN,NaN,NaN,ensembl_havana,ATAD3B-206,ATAD3B,basic,CCDS30,protein_coding,ENST00000673477,NaN,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371239,KI270713.1,ensembl,five_prime_utr,32373,32528,.,-,.,ENSG00000277475,NaN,ensembl,NaN,protein_coding,1,NA,NaN,NaN,ensembl,NaN,NaN,basic,NaN,protein_coding,ENST00000612315,NaN,1
3371240,KI270713.1,ensembl,three_prime_utr,31698,31841,.,-,.,ENSG00000277475,NaN,ensembl,NaN,protein_coding,1,NA,NaN,NaN,ensembl,NaN,NaN,basic,NaN,protein_coding,ENST00000612315,NaN,1
3371241,KI270713.1,ensembl,gene,21861,22024,.,-,.,ENSG00000275405,NaN,ensembl,NaN,snRNA,NaN,NaN,NaN,NaN,NaN,NaN,U1,NaN,NaN,NaN,NaN,NaN,1
3371242,KI270713.1,ensembl,transcript,21861,22024,.,-,.,ENSG00000275405,NaN,ensembl,NaN,snRNA,1,NA,NaN,NaN,ensembl,U1.26-201,U1,basic,NaN,snRNA,ENST00000619109,NaN,1


In [ ]:
# modify gtf file which only keep unique exon
df_exon = GTFpa[(GTFpa["feature"] == "exon")][['seqname', 'source','feature','start','end','score','strand','frame','gene_id','gene_version','gene_name','gene_source','gene_biotype','exon_number']]
df_exon[["start","end"]] = df_exon[["start","end"]].apply(pd.to_numeric)
df_exon=df_exon.sort_values(by=["seqname", "start","end"],ascending=[True,True,True])
#1. remove duplicate exon within gene which has same start and end site.
df_exon_nodup=df_exon.drop_duplicates(subset=["gene_id","start","end"],keep="first")
df_exon_nodup.head(50)

In [ ]:
#2 get unique location for exon
import math

def exon_uniq(gene):
    _df_exon=df_exon_nodup.loc[(df_exon_nodup["gene_id"] == gene)].reset_index().copy()

    # save the start/end sites in list
    _exon = []
    for i in range(_df_exon.shape[0]):
        _exon.append([_df_exon.iloc[i]["start"],_df_exon.iloc[i]["end"]])
    
    #create a unique list to store unique location
    _unq = []
    _unq.append(_exon[0])

    #get unique list of exon site:
    for i in range(1,len(_exon)):
        for j in range(0,len(_unq)):
            if _exon[i][0] >= max(map(max, _unq)):
                _unq.append(_exon[i])
            else:
                if (_unq[j][0] <= _exon[i][0] < _unq[j][1]) & (_exon[i][1] > _unq[j][1]):
                    _unq[j][1] = _exon[i][1]

    def unique(list1):
        unique_list = []
    
        for x in list1:
            if x not in unique_list:
                unique_list.append(x)
        return(unique_list)

    _unq = unique(_unq)

    #process the exon table to only keep unique exon location, double confirm if all the cases are considered.
    for k in range(_df_exon.shape[0]):
        for j in range(len(_unq)):
            if ((_unq[j][0] <= _df_exon.iloc[k]['start']) & (_unq[j][1] >= _df_exon.iloc[k]['end'])):
                _df_exon.at[k,'_start'] = _unq[j][0]
                _df_exon.at[k,'_end'] = _unq[j][1]
    
    for i in range(_df_exon.shape[0]):
        if math.isnan(_df_exon.iloc[i]['_start']) :
            print("Attention: " + _df_exon.iloc[i]['gene_id'] + ",Start=" + str(_df_exon.iloc[i]['start']) + ",end=" + str(_df_exon.iloc[i]['end']) + " is not assigned value")
    
    #only keep unique exon and clean the datasets
    _df_exon_out=_df_exon.drop_duplicates(subset=["_start","_end"],keep="first").copy()
    _df_exon_out['start'] = _df_exon_out['_start']
    _df_exon_out['end'] = _df_exon_out['_end']
    
    _df_exon_out[["start","end"]] = _df_exon_out[["start","end"]].astype('int')
    _df_exon_out = _df_exon_out.drop(columns=["_start","_end"])

    #recount exon number value
    _df_exon_out = _df_exon_out.reset_index()
    _df_exon_out['exon_number'] = _df_exon_out.index+1

    return(_df_exon_out)

#check function
exon_uniq("ENSG00000160072")
exon_uniq("ENSG00000234396")
exon_uniq("ENSG00000223972")

Run genes in batches and save each DataFrame into a separate `.pkl` file. 
A new `.pkl` file will be created for every 10,000 genes.

In [ ]:
# get the unique exon location per gene
gene_list = df_exon_nodup.gene_id.unique().tolist()
saveNum = 10000
list_of_df = list()
#run gene in batch, save each df into a pkl file
for j in range(0,int(len(gene_list)/saveNum)+1):
    #initialize output dataframe
    df_out = pd.DataFrame()
    #get the unique gene_id list 
    for i in range(j*saveNum,j*saveNum+saveNum):
        if i >= len(gene_list):
            break
        _temp = exon_uniq(gene_list[i])
        df_out = pd.concat([df_out, _temp], ignore_index=True)
        print(gene_list[i])
    list_of_df.append(df_out)
    df_out.to_pickle("df_exon_gtf_"+str(j)+".pkl")

In [ ]:
import os

pkl_files = [file for file in os.listdir('.') if file.startswith('df_exon_gtf_') and file.endswith('.pkl')]

dataframes = [pd.read_pickle(file) for file in pkl_files]
gtf_all = pd.concat(dataframes, ignore_index=True)

# Check the result
print(f"Successfully combined {len(pkl_files)} files into a single DataFrame.")
print(gtf_all.head())


In [ ]:
#check df_out has identitcal locations
_ck_out = gtf_all
# _ck_out = df_out.copy()
_ck_out['_next_start'] = _ck_out.groupby('gene_id')['start'].shift(-1)
_ck_out['_ck'] = ""
for i in range(_ck_out.shape[0]):
    if math.isnan(_ck_out['_next_start'][i]):
        _ck_out["_ck"][i] = True
    else:
        _ck_out["_ck"][i] = _ck_out['_next_start'][i] >= _ck_out['end'][i]
    
_ck_out = _ck_out[(_ck_out["_ck"] == False)]
_ck_out

In [111]:

gtf_all = gtf_all.drop(columns=['level_0','index'])
gtf_all=gtf_all.sort_values(by=["seqname", "gene_id", "start", "end"],ascending=[True,True,True,True])
gtf_all

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,gene_name,gene_source,gene_biotype,exon_number
6939,1,ensembl_havana,exon,169849631,169853772,.,-,.,ENSG00000000457,14,SCYL3,ensembl_havana,protein_coding,1
6940,1,ensembl_havana,exon,169854270,169854964,.,-,.,ENSG00000000457,14,SCYL3,ensembl_havana,protein_coding,2
6941,1,havana,exon,169855796,169855957,.,-,.,ENSG00000000457,14,SCYL3,ensembl_havana,protein_coding,3
6942,1,ensembl_havana,exon,169859041,169859212,.,-,.,ENSG00000000457,14,SCYL3,ensembl_havana,protein_coding,4
6943,1,ensembl_havana,exon,169862613,169862797,.,-,.,ENSG00000000457,14,SCYL3,ensembl_havana,protein_coding,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341415,Y,havana,exon,21250550,21250708,.,+,.,ENSG00000289707,1,NaN,havana,lncRNA,8
341416,Y,havana,exon,21253123,21253193,.,+,.,ENSG00000289707,1,NaN,havana,lncRNA,9
341417,Y,havana,exon,21255420,21257832,.,+,.,ENSG00000289707,1,NaN,havana,lncRNA,10
341902,Y,havana_tagene,exon,7274826,7275065,.,+,.,ENSG00000289826,1,NaN,havana_tagene,lncRNA,1


In [109]:
#manual check
gtf_all[(gtf_all["gene_id"] == "ENSG00000223972")]
GTFpa[(GTFpa["gene_id"] == "ENSG00000223972") & (GTFpa["feature"] == "exon")]

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,gene_name,gene_source,gene_biotype,exon_number
20024,1,havana,exon,11869,12227,.,+,.,ENSG00000223972,5,DDX11L1,havana,transcribed_unprocessed_pseudogene,1
20025,1,havana,exon,12613,12721,.,+,.,ENSG00000223972,5,DDX11L1,havana,transcribed_unprocessed_pseudogene,2
20026,1,havana,exon,12975,13052,.,+,.,ENSG00000223972,5,DDX11L1,havana,transcribed_unprocessed_pseudogene,3
20027,1,havana,exon,13221,14409,.,+,.,ENSG00000223972,5,DDX11L1,havana,transcribed_unprocessed_pseudogene,4


In [ ]:
#output to gtf file and pickle file
gtfpy.writeGTF(gtf_all,"Homo_sapiens.GRCh38.107.exon.gtf")

In [ ]:
### Convert to `gtf.pkl` File for Fast Loading in Step 2
df_mod_gtf=gtfpy.parseGTF(gtfpy.readGTF("Homo_sapiens.GRCh38.107.exon.gtf"))
df_mod_gtf.to_pickle("gtf.pkl")